Task: merge participant demographics info in `HOPE_paper2_demographics.csv` with `*_download_participant.csv` tables obtained from the Beiwe server

Why? need beiwe-id to study-id mapping in order to download data from the Beiwe server

In [69]:
import os
import glob
import pandas as pd
import yaml

# import hardcoded variables e.g., study name to ID mappings, directory paths, etc. from config file
CONFIG_PATH = "../../config/HOPE_config.yaml"
CONFIG_DIR = os.path.dirname(CONFIG_PATH)

with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)
dload_part_dir = os.path.abspath(os.path.join(CONFIG_DIR, config["download_participants_dir"]))
data_dir = os.path.abspath(os.path.join(CONFIG_DIR, config["data_dir"]))

# get all CSV files matching the naming pattern "*_download_participants.csv" inside download_participants" directory
notebook_dir = os.getcwd()

# define the relative path to the folder containing CSV files
csv_files = glob.glob(os.path.join(dload_part_dir, "*_download_participants.csv"))

In [22]:
csv_files

['/n/onnela_dp_l3/Lab/HOPE/beiwe/data/download_participants/test2_download_participants.csv',
 '/n/onnela_dp_l3/Lab/HOPE/beiwe/data/download_participants/test_download_participants.csv',
 '/n/onnela_dp_l3/Lab/HOPE/beiwe/data/download_participants/trial_download_participants.csv',
 '/n/onnela_dp_l3/Lab/HOPE/beiwe/data/download_participants/trial_phase2_download_participants.csv',
 '/n/onnela_dp_l3/Lab/HOPE/beiwe/data/download_participants/trial_phase2_passive_data_only_download_participants.csv',
 '/n/onnela_dp_l3/Lab/HOPE/beiwe/data/download_participants/troubleshooting_download_participants.csv']

In [24]:
study_name_to_id = config["study_name_to_id"]
print(study_name_to_id)

59c2b5b4388cd6715a958247
{'test': '59c2b5b4388cd6715a958247', 'test2': '5h7D9XT2vrN3BWkdcbYVNtpI', 'trial': '588224eff4d48a76f488cdfd', 'trial_phase2': '598365d5388cd66a62ac1f9e', 'trial_phase2_passive_data_only': '5a2ae1dc03d3c425ef0ea752', 'troubleshooting': '5a79f17d03d3c45080924ed4'}


In [26]:
# List to store dataframes
dataframes = []

for file_path in csv_files:
    # Extract the base filename 
    filename = os.path.basename(file_path)

    # Extract the study name from the filename before "_download_participants.csv"
    study_name = filename.replace("_download_participants.csv", "")

    # Check if the extracted study name exists in the dictionary
    if study_name in study_name_to_id:
        study_id = study_name_to_id[study_name]  # Lookup Study_ID
    else:
        print(f"Skipping {filename} (Study name '{study_name}' not found in dictionary)")
        continue  # Skip this file if study name is not in the dictionary

    # read to pandas df
    df = pd.read_csv(file_path)
    # print(df.head())
    # Ensure "Patient ID" column exists before merging
    if "Patient ID" in df.columns:
        # Add the new columns
        df["Study_ID"] = study_id
        df["Study_Name"] = study_name

        # Append the modified dataframe to the list
        dataframes.append(df)

        print(f"Loaded {filename} ({study_name}, ID: {study_id}) with {df.shape[0]} rows")
    else:
        print(f"Skipping {filename} (Missing 'Patient ID' column)")

   Created On Patient ID    Status OS Type    First Registration Date  \
0  2020-07-30   h6qsyvyf  Inactive     IOS  2020-07-30 14:00:00 (UTC)   
1  2020-05-02   k75kp9va  Inactive     IOS  2020-05-04 02:00:00 (UTC)   
2  2020-04-29   lz3g8h3d  Inactive     IOS  2020-04-29 13:00:00 (UTC)   
3  2020-05-07   sqdpv78e  Inactive     IOS  2020-05-07 17:00:00 (UTC)   

   Last Registration  Last Upload  Last Survey Download  Last Set Password  \
0                NaN          NaN                   NaN                NaN   
1                NaN          NaN                   NaN                NaN   
2                NaN          NaN                   NaN                NaN   
3                NaN          NaN                   NaN                NaN   

   Last Push Token Update  Last Device Settings Update  Last OS Version  \
0                     NaN                          NaN              NaN   
1                     NaN                          NaN              NaN   
2                 

In [39]:
dataframes

[   Created On Patient ID    Status OS Type    First Registration Date  \
 0  2020-07-30   h6qsyvyf  Inactive     IOS  2020-07-30 14:00:00 (UTC)   
 1  2020-05-02   k75kp9va  Inactive     IOS  2020-05-04 02:00:00 (UTC)   
 2  2020-04-29   lz3g8h3d  Inactive     IOS  2020-04-29 13:00:00 (UTC)   
 3  2020-05-07   sqdpv78e  Inactive     IOS  2020-05-07 17:00:00 (UTC)   
 
    Last Registration  Last Upload  Last Survey Download  Last Set Password  \
 0                NaN          NaN                   NaN                NaN   
 1                NaN          NaN                   NaN                NaN   
 2                NaN          NaN                   NaN                NaN   
 3                NaN          NaN                   NaN                NaN   
 
    Last Push Token Update  Last Device Settings Update  Last OS Version  \
 0                     NaN                          NaN              NaN   
 1                     NaN                          NaN              NaN   
 2 

In [72]:
# Merge all dfs vertically
if dataframes:
    combined_df = pd.concat(dataframes, axis=0, ignore_index=True) 
    print(f"\nFinal merged dataframe has {combined_df.shape[0]} rows and {combined_df.shape[1]} columns")

    # Save the merged DataFrame to a CSV file
    output_path = os.path.join(data_dir, "merged_participants.csv")
    combined_df.to_csv(output_path, index=False)
    print(f"Merged file saved to: {output_path}")
else:
    print("No valid CSV files to merge.")


Final merged dataframe has 158 rows and 17 columns
Merged file saved to: /n/onnela_dp_l3/Lab/HOPE/beiwe/data/merged_participants.csv


In [42]:
import pandas as pd
import re

def clean_names(df, clean_index=False):
    """
    Cleans column names (and optionally index names) by:
    - Stripping whitespace
    - Replacing spaces with underscores
    - Lowercasing
    - Removing non-alphanumeric characters (except underscore)
    """
    df = df.copy()  # optional: avoid modifying original dataframe

    df.columns = (
        df.columns.str.strip()
                  .str.replace(' ', '_')
                  .str.lower()
                  .str.replace(r"[^\w]", "", regex=True)
    )

    if clean_index:
        df.index = (
            df.index.str.strip()
                    .str.replace(' ', '_')
                    .str.lower()
                    .str.replace(r"[^\w]", "", regex=True)
        )

    return df

In [57]:
combined_df = clean_names(combined_df)

combined_df.head()

# rename Patient ID to "beiwe_id" 
combined_df.rename(columns={"Patient ID": "beiwe_id"}, inplace=True)

In [58]:
combined_df.head()

,created_on,beiwe_id,status,os_type,first_registration_date,last_registration,last_upload,last_survey_download,last_set_password,last_push_token_update,last_device_settings_update,last_os_version,app_version_code,app_version_name,last_heartbeat,study_id,study_name
0,2020-07-30,h6qsyvyf,Inactive,IOS,2020-07-30 14:00:00 (UTC),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5h7D9XT2vrN3BWkdcbYVNtpI,test2
1,2020-05-02,k75kp9va,Inactive,IOS,2020-05-04 02:00:00 (UTC),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5h7D9XT2vrN3BWkdcbYVNtpI,test2
2,2020-04-29,lz3g8h3d,Inactive,IOS,2020-04-29 13:00:00 (UTC),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5h7D9XT2vrN3BWkdcbYVNtpI,test2
3,2020-05-07,sqdpv78e,Inactive,IOS,2020-05-07 17:00:00 (UTC),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5h7D9XT2vrN3BWkdcbYVNtpI,test2
4,2020-02-07,1wlgkwod,Not Registered,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59c2b5b4388cd6715a958247,test


In [71]:
# pull demographic table
DEMO_TB_NAME = "HOPE_paper2_demographics.csv"
demo_tb_path = os.path.abspath(os.path.join(data_dir, DEMO_TB_NAME))
demo_df = pd.read_csv(demo_tb_path) 

In [60]:
demo_df.head()

,Unnamed: 0,ID,Gender,Age,Height (cm),Weight (kg),BMI,Health status (Type of cancer / Stage / Performance status),base_disease_site,Unnamed: 8,Unnamed: 9,Unnamed: 10,OS,Beiwe ID
0,0,1001,Female,64,162.6,79.4,30.00,Ovarian,White,NaN,NaN,NaN,Android,jy8yzsap
1,1,1002,Female,70,154.9,67.3,28.05,Ovarian,Other (specify),NaN,1.0,NaN,iOS,md7fnll7
2,2,1003,Female,56,165.4,62.5,22.85,Ovarian,Black or African American,NaN,1.0,NaN,iOS,wrb5oh7u
3,3,1004,Female,56,160.0,54.5,21.29,Cervical,White,NaN,2.0,NaN,iOS,wgs5rptp
4,4,1005,Female,57,157.5,68.5,27.61,Ovarian,Black or African American,NaN,1.0,NaN,iOS,q21jny47


In [61]:
demo_df = clean_names(demo_df)
demo_df.head()

,unnamed_0,id,gender,age,height_cm,weight_kg,bmi,health_status_type_of_cancer__stage__performance_status,base_disease_site,unnamed_8,unnamed_9,unnamed_10,os,beiwe_id
0,0,1001,Female,64,162.6,79.4,30.00,Ovarian,White,NaN,NaN,NaN,Android,jy8yzsap
1,1,1002,Female,70,154.9,67.3,28.05,Ovarian,Other (specify),NaN,1.0,NaN,iOS,md7fnll7
2,2,1003,Female,56,165.4,62.5,22.85,Ovarian,Black or African American,NaN,1.0,NaN,iOS,wrb5oh7u
3,3,1004,Female,56,160.0,54.5,21.29,Cervical,White,NaN,2.0,NaN,iOS,wgs5rptp
4,4,1005,Female,57,157.5,68.5,27.61,Ovarian,Black or African American,NaN,1.0,NaN,iOS,q21jny47


In [62]:
demo_df_merged= pd.merge(demo_df, combined_df, on="beiwe_id", how="inner")

In [63]:
demo_df_merged.head()

,unnamed_0,id,gender,age,height_cm,weight_kg,bmi,health_status_type_of_cancer__stage__performance_status,base_disease_site,unnamed_8,...,last_survey_download,last_set_password,last_push_token_update,last_device_settings_update,last_os_version,app_version_code,app_version_name,last_heartbeat,study_id,study_name
0,0,1001,Female,64,162.6,79.4,30.00,Ovarian,White,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,588224eff4d48a76f488cdfd,trial
1,1,1002,Female,70,154.9,67.3,28.05,Ovarian,Other (specify),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,588224eff4d48a76f488cdfd,trial
2,2,1003,Female,56,165.4,62.5,22.85,Ovarian,Black or African American,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,588224eff4d48a76f488cdfd,trial
3,3,1004,Female,56,160.0,54.5,21.29,Cervical,White,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,588224eff4d48a76f488cdfd,trial
4,4,1005,Female,57,157.5,68.5,27.61,Ovarian,Black or African American,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,588224eff4d48a76f488cdfd,trial


In [73]:
demo_df_merged.to_csv(os.path.abspath(os.path.join(data_dir, "HOPE_paper2_demographics_w_study.csv")))

In [74]:
# version just containing Beiwe_ID, Study_Name, Study_ID
demo_df_sharing = demo_df_merged[["beiwe_id", "study_name", "study_id"]]
demo_df_sharing.to_csv(os.path.abspath(os.path.join(data_dir, "HOPE_paper2_studies_and_ids_sharing.csv")))